In [34]:
import numpy as np
import pandas as pd
import json
import os
import openpyxl
import requests
from bs4 import BeautifulSoup

In [4]:
path = "ipl_teamAvsB"
data_list = []

for filename in os.listdir(path):
    # Check if the file is a JSON file
    if filename.endswith('.json'):
        # Construct the full file path
        file_path = os.path.join(path, filename)
        
        # Open and read the JSON file
        with open(file_path, 'r') as file:
            # Load the JSON data
            json_data = json.load(file)
            
            # Append the JSON data to the list
            data_list.append(json_data)

# Convert the list of JSON data to a DataFrame
df_matches = pd.DataFrame(data_list)


In [5]:
df_matches.head(5)

,meta,info,innings
0,"{'data_version': '1.0.0', 'created': '2017-04-...","{'balls_per_over': 6, 'city': 'Hyderabad', 'da...","[{'team': 'Sunrisers Hyderabad', 'overs': [{'o..."
1,"{'data_version': '1.0.0', 'created': '2017-04-...","{'balls_per_over': 6, 'city': 'Pune', 'dates':...","[{'team': 'Mumbai Indians', 'overs': [{'over':..."
2,"{'data_version': '1.0.0', 'created': '2017-04-...","{'balls_per_over': 6, 'city': 'Rajkot', 'dates...","[{'team': 'Gujarat Lions', 'overs': [{'over': ..."
3,"{'data_version': '1.0.0', 'created': '2017-04-...","{'balls_per_over': 6, 'city': 'Indore', 'dates...","[{'team': 'Rising Pune Supergiant', 'overs': [..."
4,"{'data_version': '1.0.0', 'created': '2017-04-...","{'balls_per_over': 6, 'city': 'Bengaluru', 'da...","[{'team': 'Royal Challengers Bangalore', 'over..."


In [6]:
df_matches.drop("meta",axis="columns")

,info,innings
0,"{'balls_per_over': 6, 'city': 'Hyderabad', 'da...","[{'team': 'Sunrisers Hyderabad', 'overs': [{'o..."
1,"{'balls_per_over': 6, 'city': 'Pune', 'dates':...","[{'team': 'Mumbai Indians', 'overs': [{'over':..."
2,"{'balls_per_over': 6, 'city': 'Rajkot', 'dates...","[{'team': 'Gujarat Lions', 'overs': [{'over': ..."
3,"{'balls_per_over': 6, 'city': 'Indore', 'dates...","[{'team': 'Rising Pune Supergiant', 'overs': [..."
4,"{'balls_per_over': 6, 'city': 'Bengaluru', 'da...","[{'team': 'Royal Challengers Bangalore', 'over..."
...,...,...
1019,"{'balls_per_over': 6, 'city': 'Raipur', 'dates...","[{'team': 'Delhi Daredevils', 'overs': [{'over..."
1020,"{'balls_per_over': 6, 'city': 'Bangalore', 'da...","[{'team': 'Gujarat Lions', 'overs': [{'over': ..."
1021,"{'balls_per_over': 6, 'city': 'Delhi', 'dates'...","[{'team': 'Sunrisers Hyderabad', 'overs': [{'o..."
1022,"{'balls_per_over': 6, 'city': 'Delhi', 'dates'...","[{'team': 'Gujarat Lions', 'overs': [{'over': ..."


In [7]:
sample_data = {
    "balls_per_over": [],
    "city": [],
    "dates": [],
    "match_referees": [],
    "reserve_umpires": [],
    "tv_umpires": [],
    "umpires": [],
    "winner": [],
    "player_of_match": [],
    "toss_decision": [],
    "toss_winner": [],
    "venue": [],
    "team_a": [],
    "team_b": [],
    "target_overs": 0,
    "target_runs": 0,
}

In [8]:
# Update sample_data dictionary to include target_runs and target_overs
sample_data = {
    "balls_per_over": [],
    "city": [],
    "dates": [],
    "match_referees": [],
    "reserve_umpires": [],
    "tv_umpires": [],
    "umpires": [],
    "winner": [],
    "player_of_match": [],
    "toss_decision": [],
    "toss_winner": [],
    "venue": [],
    "team_a": [],
    "team_b": [],
    "target_overs": [],
    "target_runs": []
}

# Read JSON files and populate the DataFrame
folder_path = 'ipl_teamAvsB'
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            
            # Fetch values and append to the sample_data dictionary
            sample_data["balls_per_over"].append(data["info"]["balls_per_over"])
            sample_data["city"].append(data["info"].get("city", ""))
            sample_data["dates"].append(data["info"]["dates"][0])
            sample_data["match_referees"].append(data["info"]["officials"]["match_referees"][0] if data["info"]["officials"]["match_referees"] else "")
            sample_data["reserve_umpires"].append(data["info"]["officials"].get("reserve_umpires", [""])[0])
            sample_data["tv_umpires"].append(data["info"]["officials"].get("tv_umpires", [""])[0])
            sample_data["umpires"].append(", ".join(data["info"]["officials"]["umpires"]))
            sample_data['winner'].append(data["info"]["outcome"].get('winner', ""))
            sample_data["player_of_match"].append(", ".join(data["info"].get("player_of_match", [])))
            sample_data["toss_decision"].append(data["info"]["toss"]["decision"])
            sample_data["toss_winner"].append(data["info"]["toss"]["winner"])
            sample_data["venue"].append(data["info"]["venue"])
            sample_data["team_a"].append(data["info"]["teams"][0])
            sample_data["team_b"].append(data["info"]["teams"][1])
            
            # Fetch target runs and overs from innings section
            innings_data = data.get("innings", [])
            if len(innings_data) > 1:
                target_overs = innings_data[1].get("target", {}).get("overs", 0)
                target_runs = innings_data[1].get("target", {}).get("runs", 0)
                sample_data["target_overs"].append(target_overs)
                sample_data["target_runs"].append(target_runs)
            else:
                sample_data["target_overs"].append(0)
                sample_data["target_runs"].append(0)

# Convert sample_data dictionary to DataFrame
df_matches = pd.DataFrame(sample_data)

# Display the DataFrame info
print(df_matches.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   balls_per_over   1024 non-null   int64  
 1   city             1024 non-null   object 
 2   dates            1024 non-null   object 
 3   match_referees   1024 non-null   object 
 4   reserve_umpires  1024 non-null   object 
 5   tv_umpires       1024 non-null   object 
 6   umpires          1024 non-null   object 
 7   winner           1024 non-null   object 
 8   player_of_match  1024 non-null   object 
 9   toss_decision    1024 non-null   object 
 10  toss_winner      1024 non-null   object 
 11  venue            1024 non-null   object 
 12  team_a           1024 non-null   object 
 13  team_b           1024 non-null   object 
 14  target_overs     1024 non-null   float64
 15  target_runs      1024 non-null   int64  
dtypes: float64(1), int64(2), object(13)
memory usage: 128.1+ KB


In [9]:
df_matches

,balls_per_over,city,dates,match_referees,reserve_umpires,tv_umpires,umpires,winner,player_of_match,toss_decision,toss_winner,venue,team_a,team_b,target_overs,target_runs
0,6,Hyderabad,2017-04-05,J Srinath,N Pandit,A Deshmukh,"AY Dandekar, NJ Llong",Sunrisers Hyderabad,Yuvraj Singh,field,Royal Challengers Bangalore,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Royal Challengers Bangalore,20.0,208
1,6,Pune,2017-04-06,M Nayyar,Navdeep Singh,VK Sharma,"A Nand Kishore, S Ravi",Rising Pune Supergiant,SPD Smith,field,Rising Pune Supergiant,Maharashtra Cricket Association Stadium,Rising Pune Supergiant,Mumbai Indians,20.0,185
2,6,Rajkot,2017-04-07,V Narayan Kutty,K Srinivasan,YC Barde,"Nitin Menon, CK Nandan",Kolkata Knight Riders,CA Lynn,field,Kolkata Knight Riders,Saurashtra Cricket Association Stadium,Gujarat Lions,Kolkata Knight Riders,20.0,184
3,6,Indore,2017-04-08,Chinmay Sharma,R Pandit,KN Ananthapadmanabhan,"AK Chaudhary, C Shamshuddin",Kings XI Punjab,GJ Maxwell,field,Kings XI Punjab,Holkar Cricket Stadium,Kings XI Punjab,Rising Pune Supergiant,20.0,164
4,6,Bengaluru,2017-04-08,J Srinath,Navdeep Singh,A Nand Kishore,"S Ravi, VK Sharma",Royal Challengers Bangalore,KM Jadhav,bat,Royal Challengers Bangalore,M.Chinnaswamy Stadium,Royal Challengers Bangalore,Delhi Daredevils,20.0,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,6,Raipur,2016-05-22,Chinmay Sharma,YC Barde,VK Sharma,"A Nand Kishore, BNJ Oxenford",Royal Challengers Bangalore,V Kohli,field,Royal Challengers Bangalore,Shaheed Veer Narayan Singh International Stadium,Delhi Daredevils,Royal Challengers Bangalore,20.0,139
1020,6,Bangalore,2016-05-24,RS Mahanama,A Deshmukh,BNJ Oxenford,"AK Chaudhary, HDPK Dharmasena",Royal Challengers Bangalore,AB de Villiers,field,Royal Challengers Bangalore,M Chinnaswamy Stadium,Gujarat Lions,Royal Challengers Bangalore,20.0,159
1021,6,Delhi,2016-05-25,RS Mahanama,Navdeep Singh,CK Nandan,"M Erasmus, C Shamshuddin",Sunrisers Hyderabad,MC Henriques,field,Kolkata Knight Riders,Feroz Shah Kotla,Sunrisers Hyderabad,Kolkata Knight Riders,20.0,163
1022,6,Delhi,2016-05-27,RS Mahanama,Navdeep Singh,C Shamshuddin,"M Erasmus, CK Nandan",Sunrisers Hyderabad,DA Warner,field,Sunrisers Hyderabad,Feroz Shah Kotla,Gujarat Lions,Sunrisers Hyderabad,20.0,163


In [12]:
df_matches.to_excel('match_data_2008_2023.xlsx', index=True)


In [15]:
# URL of the webpage containing the table
url = 'http://www.howstat.com/cricket/Statistics/IPL/PlayerList.asp'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table
    table = soup.find('table', class_='TableLined')

    # Initialize lists to store data
    data = []

    # Extract table rows
    rows = table.find_all('tr')

    # Iterate over rows
    for row in rows[1:]:  # Skip the header row
        # Extract table cells
        cells = row.find_all('td')
        # Extract text from cells and append to data list
        data.append([cell.get_text(strip=True) for cell in cells])

    # Convert data into a DataFrame
    df_players = pd.DataFrame(data, columns=['Name', 'Team(s)', 'Matches', 'Runs', 'Bat Avg', 'Wickets', 'Bowl Avg'])

    # Display the DataFrame
    print(df_players)
else:
    print('Failed to retrieve webpage:', response.status_code)


              Name                       Team(s) Matches  Runs Bat Avg  \
0        V R Aaron       DDV, RCB, KXI, RRO, GTI      52    50   10.00   
1       K J Abbott                           KXI       5    13   13.00   
2       S A Abbott                      RCB, SUN       3    22    7.33   
3      Abdul Samad                           SUN      34   395   19.75   
4      Y A Abdulla                           KXI      11     0           
..             ...                           ...     ...   ...     ...   
693  Yudhvir Singh                           LSG       3     1    0.50   
694   Yuvraj Singh  KXI, PWI, RCB, DDV, SUN, MIN     132  2750   24.77   
695        A Zampa                 RPS, RCB, RRO      20    15    3.00   
696       V H  Zol                           RCB       3    29   14.50   
697    D N T Zoysa                           DCH       3    11   11.00   

    Wickets Bowl Avg  
0        44    33.66  
1         2    88.50  
2         1   104.00  
3         2    56.5

In [16]:
df_players.to_excel('players_2008_2023.xlsx', index=True)


In [17]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      698 non-null    object
 1   Team(s)   698 non-null    object
 2   Matches   698 non-null    object
 3   Runs      698 non-null    object
 4   Bat Avg   698 non-null    object
 5   Wickets   698 non-null    object
 6   Bowl Avg  698 non-null    object
dtypes: object(7)
memory usage: 38.3+ KB


In [3]:
# URL of the webpage containing the table
url = 'http://www.howstat.com/cricket/Statistics/IPL/BattingMostAgg.asp?q=3'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table
    table = soup.find('table', class_='TableLined')

    # Initialize lists to store data
    data = []

    # Extract table rows
    rows = table.find_all('tr')

    # Iterate over rows
    for row in rows[1:]:  # Skip the header row
        # Extract table cells
        cells = row.find_all('td')
        # Extract text from cells and append to data list
        data.append([cell.get_text(strip=True) for cell in cells])

    # Convert data into a DataFrame
    df_most_run_for_season = pd.DataFrame(data, columns=['Player', 'Team(s)', 'Matches', 'Inns','NO','Runs','HS','100s','50s','Avg','S/R','4s','6s'])

    # Display the DataFrame
    print(df_most_run_for_season)
else:
    print('Failed to retrieve webpage:', response.status_code)


                 Player                                      Team(s) Matches  \
0             V\t Kohli                                          RCB     237   
1            S\t Dhawan                      DDV, MIN, DCH, SUN, KXI     217   
2          D A\t Warner                                     DDV, SUN     176   
3          R G\t Sharma                                     DCH, MIN     243   
4           S K\t Raina                                     CSK, GUJ     205   
5     A B\t de Villiers                                     DDV, RCB     184   
6           M S\t Dhoni                                     CSK, RPS     250   
7           C H\t Gayle                                KKR, RCB, KXI     142   
8         R V\t Uthappa                 MIN, RCB, PWI, KKR, RRO, CSK     205   
9         K D\t Karthik                 DDV, KXI, MIN, RCB, GUJ, KKR     242   
10         A M\t Rahane                 MIN, RRO, RPS, DDV, KKR, CSK     172   
11         A T\t Rayudu                 

In [4]:
df_most_run_for_season.to_excel('most_run_season_2008_2023.xlsx', index=True)


In [5]:
# URL of the webpage containing the table
url = 'http://www.howstat.com/cricket/Statistics/IPL/BattingMostAgg.asp?q=1'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table
    table = soup.find('table', class_='TableLined')

    # Initialize lists to store data
    data = []

    # Extract table rows
    rows = table.find_all('tr')

    # Iterate over rows
    for row in rows[1:]:  # Skip the header row
        # Extract table cells
        cells = row.find_all('td')
        # Extract text from cells and append to data list
        data.append([cell.get_text(strip=True) for cell in cells])

    # Convert data into a DataFrame
    df_highest_batting_average = pd.DataFrame(data, columns=['Player', 'Team(s)', 'Matches', 'Inns','NO','Runs','HS','100s','50s','Avg','S/R','4s','6s'])

    # Display the DataFrame
    print(df_highest_batting_average)
else:
    print('Failed to retrieve webpage:', response.status_code)


                 Player                  Team(s) Matches  Inns    NO  Runs  \
0         M N\t van Wyk                      KKR       5     5     2   167   
1             C\t Green                      MIN      16    16     7   452   
2          D P\t Conway                      CSK      23    22     3   924   
3           K L\t Rahul       RCB, SUN, KXI, LSG     118   109    20  4163   
4         Sai Sudharsan                      GTI      13    13     2   507   
5           A C\t Voges                      RRO       9     7     3   181   
6            H M\t Amla                      KXI      16    16     3   577   
7         Iqbal Abdulla            KKR, RRO, RCB      49    13    11    88   
8          D A\t Warner                 DDV, SUN     176   176    22  6397   
9     P D\t Collingwood                      DDV       8     7     2   203   
10        R M\t Patidar                      RCB      12    11     1   404   
11      L M P\t Simmons                      MIN      29    29  

In [6]:
df_highest_batting_average.to_excel('highest_batting_average_2008_2023.xlsx', index=True)

In [7]:
# URL of the webpage containing the table
url = 'http://www.howstat.com/cricket/Statistics/IPL/BattingMostAgg.asp?q=2'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table
    table = soup.find('table', class_='TableLined')

    # Initialize lists to store data
    data = []

    # Extract table rows
    rows = table.find_all('tr')

    # Iterate over rows
    for row in rows[1:]:  # Skip the header row
        # Extract table cells
        cells = row.find_all('td')
        # Extract text from cells and append to data list
        data.append([cell.get_text(strip=True) for cell in cells])

    # Convert data into a DataFrame
    df_highest_batting_average = pd.DataFrame(data, columns=['Player', 'Team(s)', 'Matches', 'Inns','NO','Runs','HS','100s','50s','Avg','S/R','4s','6s'])

    # Display the DataFrame
    print(df_highest_batting_average)
else:
    print('Failed to retrieve webpage:', response.status_code)


                 Player             Team(s) Matches  Inns    NO  Runs    HS  \
0           T H\t David            RCB, MIN      25    24     9   418    46   
1          L J\t Wright                 PWI       7     6     1   106    44   
2         A D\t Russell            DDV, KKR     112    96    18  2262   88*   
3           D C\t Jurel                 RRO      13    11     4   152   34*   
4           K \t Cooper                 RRO      25    14     5   116    32   
5       B C J\t Cutting       RRO, SUN, MIN      21    17     6   238   39*   
6           K\t Gowtham       RRO, KXI, LSG      35    27     9   247   33*   
7           Rashid Khan            SUN, GTI     109    52    20   443   79*   
8           H\t Klaasen       RRO, RCB, SUN      19    17     3   514   104   
9     L S\t Livingstone            RRO, KXI      32    32     4   828    94   
10         Kamran Akmal                 RRO       6     6     1   128   53*   
11           P D\t Salt                 DDV       9 

In [8]:
df_highest_batting_average.to_excel('highest_scoring_rate_2008_2023.xlsx', index=True)

In [9]:
# URL of the webpage containing the table
url = 'http://www.howstat.com/cricket/Statistics/IPL/BowlingBestAgg.asp?q=2'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table
    table = soup.find('table', class_='TableLined')

    # Initialize lists to store data
    data = []

    # Extract table rows
    rows = table.find_all('tr')

    # Iterate over rows
    for row in rows[1:]:  # Skip the header row
        # Extract table cells
        cells = row.find_all('td')
        # Extract text from cells and append to data list
        data.append([cell.get_text(strip=True) for cell in cells])

    # Convert data into a DataFrame
    df_most_wicket = pd.DataFrame(data, columns=['Player','Team(s)','Mat','Overs','Mdns','Runs','Wkts','Avg','Best','E/R','S/R','4w'])

    # Display the DataFrame
    print(df_most_wicket)
else:
    print('Failed to retrieve webpage:', response.status_code)


                 Player                  Team(s)   Mat  Overs  Mdns  Runs  \
0            Y S Chahal            MIN, RCB, RRO   145  528.5     4  4056   
1           D J J Bravo            MIN, CSK, GUJ   161  519.5     3  4360   
2            P P Chawla       KXI, KKR, CSK, MIN   181  606.4     2  4796   
3              A Mishra       DDV, DCH, SUN, LSG   161  559.5     6  4129   
4              R Ashwin  CSK, RPS, KXI, DDV, RRO   197  699.0     6  4902   
..                  ...                      ...   ...    ...   ...   ...   
56         J P Faulkner       PWI, KXI, RRO, GUJ    60  204.3     3  1778   
57               S Kaul            DDV, SUN, RCB    55  201.3     3  1739   
58            S K Warne                      RRO    55  199.0     1  1447   
59            K K Ahmed                 SUN, DDV    43  162.4     2  1402   
60  No. of Records = 60                     None  None   None  None  None   

    Wkts    Avg  Best   E/R    S/R    4w  
0    187  21.69  5/40  7.67  16.

In [12]:
df_most_wicket.to_excel('most_wicket_2008_2023.xlsx',index=True)

In [15]:
# URL of the webpage containing the table
url = 'http://www.howstat.com/cricket/Statistics/IPL/BowlingBestAgg.asp?q=4'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table
    table = soup.find('table', class_='TableLined')

    # Initialize lists to store data
    data = []

    # Extract table rows
    rows = table.find_all('tr')

    # Iterate over rows
    for row in rows[1:]:  # Skip the header row
        # Extract table cells
        cells = row.find_all('td')
        # Extract text from cells and append to data list
        data.append([cell.get_text(strip=True) for cell in cells])

    # Convert data into a DataFrame
    df_most_economical = pd.DataFrame(data, columns=['Player','Team(s)','Mat','Overs','Mdns','Runs','Wkts','Avg','Best','E/R','S/R','4w'])

    # Display the DataFrame
    print(df_most_economical)
else:
    print('Failed to retrieve webpage:', response.status_code)


                 Player                  Team(s)   Mat  Overs  Mdns  Runs  \
0              A Kumble                      RCB    42  160.5     1  1058   
1           G D McGrath                      DDV    14   54.0     2   357   
2        M Muralitharan            CSK, KTK, RCB    66  254.0     1  1685   
3           Rashid Khan                 SUN, GTI   109  432.5     4  2885   
4            S P Narine                      KKR   162  624.1     3  4203   
..                  ...                      ...   ...    ...   ...   ...   
56             M Morkel            RRO, DDV, KKR    70  271.3     3  2089   
57     N M Coulter-Nile       MIN, DDV, KKR, RRO    39  142.5     1  1100   
58       B W Hilfenhaus                      CSK    17   62.0     2   479   
59              P Kumar  RCB, KXI, MIN, SUN, GUJ   119  420.4    14  3251   
60  No. of Records = 60                     None  None   None  None  None   

    Wkts    Avg  Best   E/R    S/R    4w  
0     45  23.51   5/5  6.58  21.

In [16]:
df_most_economical.to_excel('most_economical_2008_2023.xlsx',index=True)

In [17]:
# converting xlsx to csv for size reduction and faster processing

excel_folder_path = 'scraped_datasets'

# Path to the directory where CSV files will be saved
csv_folder_path = 'csv_datasets'

# Create the output folder if it doesn't exist
os.makedirs(csv_folder_path, exist_ok=True)

# Iterate over each file in the folder
for filename in os.listdir(excel_folder_path):
    if filename.endswith('.xlsx'):
        # Construct full paths for input and output files
        excel_file_path = os.path.join(excel_folder_path, filename)
        csv_file_path = os.path.join(csv_folder_path, filename[:-5] + '.csv')  # Replace .xlsx with .csv

        # Read the Excel file into a DataFrame
        df = pd.read_excel(excel_file_path)

        # Save the DataFrame to a CSV file
        df.to_csv(csv_file_path, index=False)

        print(f"Conversion completed for {filename}")

print("All conversions completed successfully.")


Conversion completed for highest_batting_average_2008_2023.xlsx
Conversion completed for highest_scoring_rate_2008_2023.xlsx
Conversion completed for match_data_2008_2023.xlsx
Conversion completed for most_economical_2008_2023.xlsx
Conversion completed for most_run_season_2008_2023.xlsx
Conversion completed for most_wicket_2008_2023.xlsx
Conversion completed for players_2008_2023.xlsx
All conversions completed successfully.


In [16]:
folder_path = 'ipl_teamAvsB'
sample_data = {
    "season": [],
    "balls_per_over": [],
    "city": [],
    "dates": [],
    "match_referees": [],
    "reserve_umpires": [],
    "tv_umpires": [],
    "umpire1": [],
    "umpire2": [],
    "winner": [],
    "result": [],
    "player_of_match": [],
    "toss_decision": [],
    "toss_winner": [],
    "dl_applied": [],
    "venue": [],
    "team_a": [],
    "team_b": [],
    "target_overs": [],
    "target_runs": [],
    "win_by_runs": [],
    "win_by_wickets": [],
}

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            
            sample_data["balls_per_over"].append(data["info"]["balls_per_over"])
            sample_data["city"].append(data["info"].get("city", ""))
            sample_data["dates"].append(data["info"]["dates"][0])
            sample_data["match_referees"].append(data["info"]["officials"]["match_referees"][0] if data["info"]["officials"]["match_referees"] else "")
            sample_data["reserve_umpires"].append(data["info"]["officials"].get("reserve_umpires", [""])[0])
            sample_data["tv_umpires"].append(data["info"]["officials"].get("tv_umpires", [""])[0])
            sample_data['winner'].append(data["info"]["outcome"].get('winner', ""))
            sample_data["player_of_match"].append(", ".join(data["info"].get("player_of_match", [])))
            sample_data["toss_decision"].append(data["info"]["toss"]["decision"])
            sample_data["toss_winner"].append(data["info"]["toss"]["winner"])
            sample_data["venue"].append(data["info"]["venue"])
            sample_data["team1"].append(data["info"]["teams"][0])
            sample_data["team2"].append(data["info"]["teams"][1])
            
            umpires = data["info"]["officials"]["umpires"]
            if len(umpires) >= 2:
                sample_data["umpire1"].append(umpires[0])
                sample_data["umpire2"].append(umpires[1])
            else:
                sample_data["umpire1"].append("")
                sample_data["umpire2"].append("")

            innings_data = data.get("innings", [])
            if len(innings_data) > 1:
                target_overs = innings_data[1].get("target", {}).get("overs", 0)
                target_runs = innings_data[1].get("target", {}).get("runs", 0)
                sample_data["target_overs"].append(target_overs)
                sample_data["target_runs"].append(target_runs)
            else:
                sample_data["target_overs"].append(0)
                sample_data["target_runs"].append(0)
            
            win_by_runs = data["info"]["outcome"].get("by", {}).get("runs", 0)
            win_by_wickets = data["info"]["outcome"].get("by", {}).get("wickets", 0)
            sample_data["win_by_runs"].append(win_by_runs)
            sample_data["win_by_wickets"].append(win_by_wickets)

            method = data["info"]["outcome"].get("method", "")
            if method == "D/L":
                sample_data["dl_applied"].append(1)
            else:
                sample_data["dl_applied"].append(0)

            result = data["info"]["outcome"].get("result", "normal")
            sample_data["result"].append(result)

            # Fetch the date and append to the season column
            date = data["info"]["dates"][0]
            year = pd.to_datetime(date).year
            sample_data["season"].append(f"IPL-{year}")

# Convert sample_data dictionary to DataFrame
df_matches = pd.DataFrame(sample_data)

# Display the DataFrame info
print(df_matches.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   season           1024 non-null   object 
 1   balls_per_over   1024 non-null   int64  
 2   city             1024 non-null   object 
 3   dates            1024 non-null   object 
 4   match_referees   1024 non-null   object 
 5   reserve_umpires  1024 non-null   object 
 6   tv_umpires       1024 non-null   object 
 7   umpire1          1024 non-null   object 
 8   umpire2          1024 non-null   object 
 9   winner           1024 non-null   object 
 10  result           1024 non-null   object 
 11  player_of_match  1024 non-null   object 
 12  toss_decision    1024 non-null   object 
 13  toss_winner      1024 non-null   object 
 14  dl_applied       1024 non-null   int64  
 15  venue            1024 non-null   object 
 16  team_a           1024 non-null   object 
 17  team_b        

In [17]:
df_matches.head(5)

,season,balls_per_over,city,dates,match_referees,reserve_umpires,tv_umpires,umpire1,umpire2,winner,...,toss_decision,toss_winner,dl_applied,venue,team_a,team_b,target_overs,target_runs,win_by_runs,win_by_wickets
0,IPL-2017,6,Hyderabad,2017-04-05,J Srinath,N Pandit,A Deshmukh,AY Dandekar,NJ Llong,Sunrisers Hyderabad,...,field,Royal Challengers Bangalore,0,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Royal Challengers Bangalore,20.0,208,35,0
1,IPL-2017,6,Pune,2017-04-06,M Nayyar,Navdeep Singh,VK Sharma,A Nand Kishore,S Ravi,Rising Pune Supergiant,...,field,Rising Pune Supergiant,0,Maharashtra Cricket Association Stadium,Rising Pune Supergiant,Mumbai Indians,20.0,185,0,7
2,IPL-2017,6,Rajkot,2017-04-07,V Narayan Kutty,K Srinivasan,YC Barde,Nitin Menon,CK Nandan,Kolkata Knight Riders,...,field,Kolkata Knight Riders,0,Saurashtra Cricket Association Stadium,Gujarat Lions,Kolkata Knight Riders,20.0,184,0,10
3,IPL-2017,6,Indore,2017-04-08,Chinmay Sharma,R Pandit,KN Ananthapadmanabhan,AK Chaudhary,C Shamshuddin,Kings XI Punjab,...,field,Kings XI Punjab,0,Holkar Cricket Stadium,Kings XI Punjab,Rising Pune Supergiant,20.0,164,0,6
4,IPL-2017,6,Bengaluru,2017-04-08,J Srinath,Navdeep Singh,A Nand Kishore,S Ravi,VK Sharma,Royal Challengers Bangalore,...,bat,Royal Challengers Bangalore,0,M.Chinnaswamy Stadium,Royal Challengers Bangalore,Delhi Daredevils,20.0,158,15,0


In [13]:
df_matches['result'] 

3

In [18]:
df_matches.to_csv('matches_2008_2023.csv')
df_matches.to_excel('matches_2008_2023.xlsx')

In [116]:
# Initialize sample_data dictionary
sample_data = {
    "match_id": [],
    "inning": [],
    "batting_team": [],
    "bowling_team": [],
    "over": [],
    "ball": [],
    "batsman": [],
    "non_striker": [],
    "bowler": [],
    "is_super_over": [],
    "wide_runs": [],
    "bye_runs": [],
    "legbye_runs": [],
    "noball_runs": [],
    "penalty_runs": [],
    "batsman_runs": [],
    "extra_runs": [],
    "total_runs": [],
    "player_dismissed": [],
    "dismissal_kind": [],
    "fielder": [],
}

match_counter = 1

with open('1082625.json', 'r') as file:
    data = json.load(file)
    
    # Fetch teams' names
    team_a = data["info"]["teams"][0]
    team_b = data["info"]["teams"][1]
    
    # Initialize match counter
    match_counter = 1
    
    # Iterate through innings
    for inning_index, inning in enumerate(data["innings"], start=1):
        # Fetch batting and bowling teams
        batting_team = inning["team"]
        bowling_team = team_a if inning["team"] == team_b else team_b
        
        
        # Iterate through overs
        for over_info in inning["overs"]:
            over_number = over_info["over"] + 1
            ball_count = 1
            
            
            # Check for the presence of "super_over"
            is_super_over = 1 if "super_over" in inning else 0
            
            # Iterate through deliveries
            for ball_info in over_info["deliveries"]:
                ball_number = ball_count
                batsman = ball_info["batter"]
                non_striker = ball_info["non_striker"]
                bowler = ball_info["bowler"]
                
                # Fetch runs information
                runs = ball_info["runs"]
                batsman_runs = runs.get("batter", 0)
                extra_runs = runs.get("extras", 0)
                total_runs = runs.get("total", 0)
                
                # Fetch dismissal information
                player_dismissed = ball_info.get("player_dismissed", "")
                dismissal_kind = ball_info.get("dismissal_kind", "")
                fielder = ball_info.get("fielder", "")
                
                # Append data to the sample_data dictionary
                sample_data["match_id"].append(match_counter)
                sample_data["inning"].append(inning_index)
                sample_data["batting_team"].append(batting_team)
                sample_data["bowling_team"].append(bowling_team)
                sample_data["over"].append(over_number)
                sample_data["ball"].append(ball_number)
                sample_data["batsman"].append(batsman)
                sample_data["non_striker"].append(non_striker)
                sample_data["bowler"].append(bowler)
                sample_data["is_super_over"].append(is_super_over)
                sample_data["wide_runs"].append(extra_runs if "wides" in runs else 0)
                sample_data["bye_runs"].append(extra_runs if "byes" in runs else 0)
                sample_data["legbye_runs"].append(extra_runs if "legbyes" in runs else 0)
                sample_data["noball_runs"].append(extra_runs if "noballs" in runs else 0)
                sample_data["penalty_runs"].append(extra_runs if "penalty" in runs else 0)
                sample_data["batsman_runs"].append(batsman_runs)
                sample_data["extra_runs"].append(extra_runs)
                sample_data["total_runs"].append(total_runs)
                sample_data["player_dismissed"].append(player_dismissed)
                sample_data["dismissal_kind"].append(dismissal_kind)
                sample_data["fielder"].append(fielder)

                ball_count += 1

        # Increment match counter when all innings are completed in a JSON file
        match_counter += 1
       

# Convert sample_data dictionary to DataFrame
df_test = pd.DataFrame(sample_data)
df_test


,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Gujarat Lions,Mumbai Indians,1,1,Ishan Kishan,BB McCullum,MJ McClenaghan,0,...,0,0,0,0,0,1,1,,,
1,1,1,Gujarat Lions,Mumbai Indians,1,2,Ishan Kishan,BB McCullum,MJ McClenaghan,0,...,0,0,0,0,0,0,0,,,
2,1,1,Gujarat Lions,Mumbai Indians,1,3,Ishan Kishan,BB McCullum,MJ McClenaghan,0,...,0,0,0,0,0,0,0,,,
3,1,1,Gujarat Lions,Mumbai Indians,1,4,Ishan Kishan,BB McCullum,MJ McClenaghan,0,...,0,0,0,0,4,0,4,,,
4,1,1,Gujarat Lions,Mumbai Indians,1,5,Ishan Kishan,BB McCullum,MJ McClenaghan,0,...,0,0,0,0,0,0,0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,4,4,Gujarat Lions,Mumbai Indians,1,4,BB McCullum,AJ Finch,JJ Bumrah,1,...,0,0,0,0,0,0,0,,,
259,4,4,Gujarat Lions,Mumbai Indians,1,5,BB McCullum,AJ Finch,JJ Bumrah,1,...,0,0,0,0,0,1,1,,,
260,4,4,Gujarat Lions,Mumbai Indians,1,6,AJ Finch,BB McCullum,JJ Bumrah,1,...,0,0,0,0,0,0,0,,,
261,4,4,Gujarat Lions,Mumbai Indians,1,7,AJ Finch,BB McCullum,JJ Bumrah,1,...,0,0,0,0,1,0,1,,,


In [117]:
# df_test[df_test['player_dismissed'] != '']
df_test[df_test['is_super_over']>0]

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
250,3,3,Mumbai Indians,Gujarat Lions,1,1,JC Buttler,KA Pollard,JP Faulkner,1,...,0,0,0,0,1,0,1,,,
251,3,3,Mumbai Indians,Gujarat Lions,1,2,KA Pollard,JC Buttler,JP Faulkner,1,...,0,0,0,0,4,0,4,,,
252,3,3,Mumbai Indians,Gujarat Lions,1,3,KA Pollard,JC Buttler,JP Faulkner,1,...,0,0,0,0,6,0,6,,,
253,3,3,Mumbai Indians,Gujarat Lions,1,4,KA Pollard,JC Buttler,JP Faulkner,1,...,0,0,0,0,0,0,0,,,
254,3,3,Mumbai Indians,Gujarat Lions,1,5,JC Buttler,RG Sharma,JP Faulkner,1,...,0,0,0,0,0,0,0,,,
255,4,4,Gujarat Lions,Mumbai Indians,1,1,AJ Finch,BB McCullum,JJ Bumrah,1,...,0,0,0,0,0,1,1,,,
256,4,4,Gujarat Lions,Mumbai Indians,1,2,AJ Finch,BB McCullum,JJ Bumrah,1,...,0,0,0,0,0,1,1,,,
257,4,4,Gujarat Lions,Mumbai Indians,1,3,BB McCullum,AJ Finch,JJ Bumrah,1,...,0,0,0,0,0,1,1,,,
258,4,4,Gujarat Lions,Mumbai Indians,1,4,BB McCullum,AJ Finch,JJ Bumrah,1,...,0,0,0,0,0,0,0,,,
259,4,4,Gujarat Lions,Mumbai Indians,1,5,BB McCullum,AJ Finch,JJ Bumrah,1,...,0,0,0,0,0,1,1,,,


In [67]:
df_deliveries = pd.read_csv('deliveries.csv')

In [36]:
df_deliveries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179078 entries, 0 to 179077
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   match_id          179078 non-null  int64 
 1   inning            179078 non-null  int64 
 2   batting_team      179078 non-null  object
 3   bowling_team      179078 non-null  object
 4   over              179078 non-null  int64 
 5   ball              179078 non-null  int64 
 6   batsman           179078 non-null  object
 7   non_striker       179078 non-null  object
 8   bowler            179078 non-null  object
 9   is_super_over     179078 non-null  int64 
 10  wide_runs         179078 non-null  int64 
 11  bye_runs          179078 non-null  int64 
 12  legbye_runs       179078 non-null  int64 
 13  noball_runs       179078 non-null  int64 
 14  penalty_runs      179078 non-null  int64 
 15  batsman_runs      179078 non-null  int64 
 16  extra_runs        179078 non-null  int

In [78]:
unique_innings = df_deliveries['ball'].unique()

print("Unique values in column 'innings':")
for value in unique_innings:
    print(value)

Unique values in column 'innings':
1
2
3
4
5
6
7
8
9


In [104]:
df_deliveries[(df_deliveries['match_id']==34) & (df_deliveries['is_super_over']>0)]


,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
8092,34,3,Mumbai Indians,Gujarat Lions,1,1,JC Buttler,KA Pollard,JP Faulkner,1,...,0,0,0,0,1,0,1,NaN,NaN,NaN
8093,34,3,Mumbai Indians,Gujarat Lions,1,2,KA Pollard,JC Buttler,JP Faulkner,1,...,0,0,0,0,4,0,4,NaN,NaN,NaN
8094,34,3,Mumbai Indians,Gujarat Lions,1,3,KA Pollard,JC Buttler,JP Faulkner,1,...,0,0,0,0,6,0,6,NaN,NaN,NaN
8095,34,3,Mumbai Indians,Gujarat Lions,1,4,KA Pollard,JC Buttler,JP Faulkner,1,...,0,0,0,0,0,0,0,KA Pollard,caught,AJ Finch
8096,34,3,Mumbai Indians,Gujarat Lions,1,5,JC Buttler,RG Sharma,JP Faulkner,1,...,0,0,0,0,0,0,0,JC Buttler,caught,Ishan Kishan
8097,34,4,Gujarat Lions,Mumbai Indians,1,1,AJ Finch,BB McCullum,JJ Bumrah,1,...,0,0,1,0,0,1,1,NaN,NaN,NaN
8098,34,4,Gujarat Lions,Mumbai Indians,1,2,AJ Finch,BB McCullum,JJ Bumrah,1,...,0,1,0,0,0,1,1,NaN,NaN,NaN
8099,34,4,Gujarat Lions,Mumbai Indians,1,3,BB McCullum,AJ Finch,JJ Bumrah,1,...,0,0,0,0,0,1,1,NaN,NaN,NaN
8100,34,4,Gujarat Lions,Mumbai Indians,1,4,BB McCullum,AJ Finch,JJ Bumrah,1,...,0,0,0,0,0,0,0,NaN,NaN,NaN
8101,34,4,Gujarat Lions,Mumbai Indians,1,5,BB McCullum,AJ Finch,JJ Bumrah,1,...,1,0,0,0,0,1,1,NaN,NaN,NaN


In [123]:
sample_data1 = {
    "match_id": [],
    "inning": [],
    "batting_team": [],
    "bowling_team": [],
    "over": [],
    "ball": [],
    "batsman": [],
    "non_striker": [],
    "bowler": [],
    "is_super_over": [],
    "wide_runs": [],
    "bye_runs": [],
    "legbye_runs": [],
    "noball_runs": [],
    "penalty_runs": [],
    "batsman_runs": [],
    "extra_runs": [],
    "total_runs": [],
    "player_dismissed": [],
    "dismissal_kind": [],
    "fielder": [],
}

sample_data2 = {
    "match_id": [],
    "season": [],
    "balls_per_over": [],
    "city": [],
    "dates": [],
    "match_referees": [],
    "reserve_umpires": [],
    "tv_umpires": [],
    "umpire1": [],
    "umpire2": [],
    "winner": [],
    "result": [],
    "player_of_match": [],
    "toss_decision": [],
    "toss_winner": [],
    "dl_applied": [],
    "venue": [],
    "team_a": [],
    "team_b": [],
    "target_overs": [],
    "target_runs": [],
    "win_by_runs": [],
    "win_by_wickets": [],
}

# Set the folder path
folder_path = 'ipl_teamAvsB'

# Initialize match counter
match_counter = 1

# Iterate through files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        # Construct the file path
        file_path = os.path.join(folder_path, filename)
        
        # Open the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)

            # Append match_id
            sample_data2["match_id"].append(match_counter)
            
            # Append other data
            sample_data2["balls_per_over"].append(data["info"]["balls_per_over"])
            sample_data2["city"].append(data["info"].get("city", ""))
            sample_data2["dates"].append(data["info"]["dates"][0])
            sample_data2["match_referees"].append(data["info"]["officials"]["match_referees"][0] if data["info"]["officials"]["match_referees"] else "")
            sample_data2["reserve_umpires"].append(data["info"]["officials"].get("reserve_umpires", [""])[0])
            sample_data2["tv_umpires"].append(data["info"]["officials"].get("tv_umpires", [""])[0])
            sample_data2['winner'].append(data["info"]["outcome"].get('winner', ""))
            sample_data2["player_of_match"].append(", ".join(data["info"].get("player_of_match", [])))
            sample_data2["toss_decision"].append(data["info"]["toss"]["decision"])
            sample_data2["toss_winner"].append(data["info"]["toss"]["winner"])
            sample_data2["venue"].append(data["info"]["venue"])
            sample_data2["team_a"].append(data["info"]["teams"][0])
            sample_data2["team_b"].append(data["info"]["teams"][1])
            
            umpires = data["info"]["officials"]["umpires"]
            if len(umpires) >= 2:
                sample_data2["umpire1"].append(umpires[0])
                sample_data2["umpire2"].append(umpires[1])
            else:
                sample_data2["umpire1"].append("")
                sample_data2["umpire2"].append("")

            innings_data = data.get("innings", [])
            if len(innings_data) > 1:
                target_overs = innings_data[1].get("target", {}).get("overs", 0)
                target_runs = innings_data[1].get("target", {}).get("runs", 0)
                sample_data2["target_overs"].append(target_overs)
                sample_data2["target_runs"].append(target_runs)
            else:
                sample_data2["target_overs"].append(0)
                sample_data2["target_runs"].append(0)
            
            win_by_runs = data["info"]["outcome"].get("by", {}).get("runs", 0)
            win_by_wickets = data["info"]["outcome"].get("by", {}).get("wickets", 0)
            sample_data2["win_by_runs"].append(win_by_runs)
            sample_data2["win_by_wickets"].append(win_by_wickets)

            method = data["info"]["outcome"].get("method", "")
            if method == "D/L":
                sample_data2["dl_applied"].append(1)
            else:
                sample_data2["dl_applied"].append(0)

            result = data["info"]["outcome"].get("result", "normal")
            sample_data2["result"].append(result)

            # Fetch the date and append to the season column
            date = data["info"]["dates"][0]
            year = pd.to_datetime(date).year
            sample_data2["season"].append(f"IPL-{year}")

            # Fetch teams' names
            team_a = data["info"]["teams"][0]
            team_b = data["info"]["teams"][1]

            # Iterate through innings
            for inning_index, inning in enumerate(data["innings"], start=1):
                # Fetch batting and bowling teams
                batting_team = inning["team"]
                bowling_team = team_a if inning["team"] == team_b else team_b
                
                
                # Iterate through overs
                for over_info in inning["overs"]:
                    over_number = over_info["over"] + 1
                    ball_count = 1
                    
                    # Check for the presence of "super_over"
                    is_super_over = 1 if "super_over" in inning else 0
                    
                    # Iterate through deliveries
                    for ball_info in over_info["deliveries"]:
                        ball_number = ball_count
                        batsman = ball_info["batter"]
                        non_striker = ball_info["non_striker"]
                        bowler = ball_info["bowler"]
                        
                        # Fetch runs information
                        runs = ball_info["runs"]
                        batsman_runs = runs.get("batter", 0)
                        extra_runs = runs.get("extras", 0)
                        total_runs = runs.get("total", 0)
                        
                        wickets = ball_info.get("wickets", [])
                        if wickets:
                            wicket = wickets[0]  # Assuming only one wicket per delivery for simplicity
                            player_dismissed = wicket["player_out"]
                            dismissal_kind = wicket["kind"]
                            fielder = wicket.get("fielders", [{}])[0].get("name", "")
                        else:
                            player_dismissed = ""
                            dismissal_kind = ""
                            fielder = ""
                        
                        # Append data to the sample_data dictionary
                        sample_data1["match_id"].append(match_counter)
                        sample_data1["inning"].append(inning_index)
                        sample_data1["batting_team"].append(batting_team)
                        sample_data1["bowling_team"].append(bowling_team)
                        sample_data1["over"].append(over_number)
                        sample_data1["ball"].append(ball_number)
                        sample_data1["batsman"].append(batsman)
                        sample_data1["non_striker"].append(non_striker)
                        sample_data1["bowler"].append(bowler)
                        sample_data1["is_super_over"].append(is_super_over)
                        sample_data1["wide_runs"].append(extra_runs if "wides" in runs else 0)
                        sample_data1["bye_runs"].append(extra_runs if "byes" in runs else 0)
                        sample_data1["legbye_runs"].append(extra_runs if "legbyes" in runs else 0)
                        sample_data1["noball_runs"].append(extra_runs if "noballs" in runs else 0)
                        sample_data1["penalty_runs"].append(extra_runs if "penalty" in runs else 0)
                        sample_data1["batsman_runs"].append(batsman_runs)
                        sample_data1["extra_runs"].append(extra_runs)
                        sample_data1["total_runs"].append(total_runs)
                        sample_data1["player_dismissed"].append(player_dismissed)
                        sample_data1["dismissal_kind"].append(dismissal_kind)
                        sample_data1["fielder"].append(fielder)

                        ball_count += 1
            
            # Increment the match counter
            match_counter += 1

df_match_test = pd.DataFrame(sample_data2)
df_deliveries_test = pd.DataFrame(sample_data1)


In [125]:
df_match_test

,match_id,season,balls_per_over,city,dates,match_referees,reserve_umpires,tv_umpires,umpire1,umpire2,...,toss_decision,toss_winner,dl_applied,venue,team_a,team_b,target_overs,target_runs,win_by_runs,win_by_wickets
0,1,IPL-2017,6,Hyderabad,2017-04-05,J Srinath,N Pandit,A Deshmukh,AY Dandekar,NJ Llong,...,field,Royal Challengers Bangalore,0,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Royal Challengers Bangalore,20.0,208,35,0
1,2,IPL-2017,6,Pune,2017-04-06,M Nayyar,Navdeep Singh,VK Sharma,A Nand Kishore,S Ravi,...,field,Rising Pune Supergiant,0,Maharashtra Cricket Association Stadium,Rising Pune Supergiant,Mumbai Indians,20.0,185,0,7
2,3,IPL-2017,6,Rajkot,2017-04-07,V Narayan Kutty,K Srinivasan,YC Barde,Nitin Menon,CK Nandan,...,field,Kolkata Knight Riders,0,Saurashtra Cricket Association Stadium,Gujarat Lions,Kolkata Knight Riders,20.0,184,0,10
3,4,IPL-2017,6,Indore,2017-04-08,Chinmay Sharma,R Pandit,KN Ananthapadmanabhan,AK Chaudhary,C Shamshuddin,...,field,Kings XI Punjab,0,Holkar Cricket Stadium,Kings XI Punjab,Rising Pune Supergiant,20.0,164,0,6
4,5,IPL-2017,6,Bengaluru,2017-04-08,J Srinath,Navdeep Singh,A Nand Kishore,S Ravi,VK Sharma,...,bat,Royal Challengers Bangalore,0,M.Chinnaswamy Stadium,Royal Challengers Bangalore,Delhi Daredevils,20.0,158,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,1020,IPL-2016,6,Raipur,2016-05-22,Chinmay Sharma,YC Barde,VK Sharma,A Nand Kishore,BNJ Oxenford,...,field,Royal Challengers Bangalore,0,Shaheed Veer Narayan Singh International Stadium,Delhi Daredevils,Royal Challengers Bangalore,20.0,139,0,6
1020,1021,IPL-2016,6,Bangalore,2016-05-24,RS Mahanama,A Deshmukh,BNJ Oxenford,AK Chaudhary,HDPK Dharmasena,...,field,Royal Challengers Bangalore,0,M Chinnaswamy Stadium,Gujarat Lions,Royal Challengers Bangalore,20.0,159,0,4
1021,1022,IPL-2016,6,Delhi,2016-05-25,RS Mahanama,Navdeep Singh,CK Nandan,M Erasmus,C Shamshuddin,...,field,Kolkata Knight Riders,0,Feroz Shah Kotla,Sunrisers Hyderabad,Kolkata Knight Riders,20.0,163,22,0
1022,1023,IPL-2016,6,Delhi,2016-05-27,RS Mahanama,Navdeep Singh,C Shamshuddin,M Erasmus,CK Nandan,...,field,Sunrisers Hyderabad,0,Feroz Shah Kotla,Gujarat Lions,Sunrisers Hyderabad,20.0,163,0,4


In [126]:
df_deliveries_test

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,,,
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,,,
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,,,
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,,,
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243812,1024,2,Royal Challengers Bangalore,Sunrisers Hyderabad,20,2,Sachin Baby,CJ Jordan,B Kumar,0,...,0,0,0,0,2,0,2,,,
243813,1024,2,Royal Challengers Bangalore,Sunrisers Hyderabad,20,3,Sachin Baby,CJ Jordan,B Kumar,0,...,0,0,0,0,0,0,0,CJ Jordan,run out,NV Ojha
243814,1024,2,Royal Challengers Bangalore,Sunrisers Hyderabad,20,4,Iqbal Abdulla,Sachin Baby,B Kumar,0,...,0,0,0,0,0,1,1,,,
243815,1024,2,Royal Challengers Bangalore,Sunrisers Hyderabad,20,5,Sachin Baby,Iqbal Abdulla,B Kumar,0,...,0,0,0,0,1,0,1,,,


In [127]:
df_match_test[df_match_test['match_id']==34]

,match_id,season,balls_per_over,city,dates,match_referees,reserve_umpires,tv_umpires,umpire1,umpire2,...,toss_decision,toss_winner,dl_applied,venue,team_a,team_b,target_overs,target_runs,win_by_runs,win_by_wickets
33,34,IPL-2017,6,Rajkot,2017-04-29,M Nayyar,N Pandit,A Nand Kishore,AK Chaudhary,CB Gaffaney,...,bat,Gujarat Lions,0,Saurashtra Cricket Association Stadium,Gujarat Lions,Mumbai Indians,20.0,154,0,0


In [128]:
df_deliveries_test[df_deliveries_test['match_id']==34]

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
7842,34,1,Gujarat Lions,Mumbai Indians,1,1,Ishan Kishan,BB McCullum,MJ McClenaghan,0,...,0,0,0,0,0,1,1,,,
7843,34,1,Gujarat Lions,Mumbai Indians,1,2,Ishan Kishan,BB McCullum,MJ McClenaghan,0,...,0,0,0,0,0,0,0,,,
7844,34,1,Gujarat Lions,Mumbai Indians,1,3,Ishan Kishan,BB McCullum,MJ McClenaghan,0,...,0,0,0,0,0,0,0,,,
7845,34,1,Gujarat Lions,Mumbai Indians,1,4,Ishan Kishan,BB McCullum,MJ McClenaghan,0,...,0,0,0,0,4,0,4,,,
7846,34,1,Gujarat Lions,Mumbai Indians,1,5,Ishan Kishan,BB McCullum,MJ McClenaghan,0,...,0,0,0,0,0,0,0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8100,34,4,Gujarat Lions,Mumbai Indians,1,4,BB McCullum,AJ Finch,JJ Bumrah,1,...,0,0,0,0,0,0,0,,,
8101,34,4,Gujarat Lions,Mumbai Indians,1,5,BB McCullum,AJ Finch,JJ Bumrah,1,...,0,0,0,0,0,1,1,,,
8102,34,4,Gujarat Lions,Mumbai Indians,1,6,AJ Finch,BB McCullum,JJ Bumrah,1,...,0,0,0,0,0,0,0,,,
8103,34,4,Gujarat Lions,Mumbai Indians,1,7,AJ Finch,BB McCullum,JJ Bumrah,1,...,0,0,0,0,1,0,1,,,


In [129]:
df_deliveries_test.to_csv('deliveries_2008_2023.csv',index=False)
df_match_test.to_csv('match_2008_2023.csv',index = False)